In [ ]:
from pyspark.sql import functions as fns
from pyspark import  SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
import json
with open("table_names.json", "r") as f:
    table_names = json.load(f)

In [ ]:
# Want to partition based on filename.

for k,v in table_names.items():
    # Some tables e.g. id 30 have very few records (possibly none?)
    print(v["filename"])
    
    # Load in Spark schema so we read with the correct datatypes
    with open("spark_schemas/{}.json".format(v["tablename"]), "r") as f:
        jsonschema = json.load(f)
    schema = StructType.fromJson(jsonschema)
    try:
        df =  spark.read.csv("s3a://alpha-everyone/deleteathenaout/outtemp/first_col={}".format(v["partition"]), schema=schema)
    except AnalysisException:
        continue

    
    headings = v["headings"]
    headings = headings + ["filename"]
    df = df.toDF(*headings)
    
    df = df.repartition(100, "filename")  # 100 partitions based on file name.  Note that this is actually a maximum of 100 partitions.  If there are <100 file names, there'll be a file per file name.

    df.write.parquet("s3a://alpha-everyone/deleteathenaout/abpparquet/{}".format(v["tablename"]), mode="overwrite")
    